In [49]:
import pandas as pd
import requests
import numpy as np
import cn2an

# output_file_name = "data-1-year" 
output_file_name = "data-10-year" 
urls = {
    'data-1-year' :"https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/fe810aa29f305b7dce1e6271556bdbf1?q=VTJGc2RHVmtYMTg2Tmo2TDdZUktQcUwyT3p5TEdYRU9HdUJBSFFNQzRhaXJvNlVzYmFQd052dndYR1A0c1lOVE5ZVUxGU2tyRU5WWC9yVnB2U0R1bGg4YlFCdnlhSWNuL3p1SUdjakxUaFJ5Vk1mV2kzSm04QU03V0FISmdzYXVoYUUvaS9pcGY3Q3NMWXpTN0tubzhXc1g4VGY0QmRiN3A5dUthSHhlNlhwQ0VIdStLaFA5aUJvNWkxWjYxaU52cXZ2cjY1aDhheG9FUitxOFdGSzlxQjE0OEdPeTdCTmVHRCswZUhnOElxZ0Z0dGIxVExqSTNMYjBTRVN2VGdacXk0MS9sQnlzaG0vQWo1eXdWNkZrN0Q2b2ViTTZUWmVaZWh3aWNiQzhBUGZCSGpPcWZldG1CVkQxUzRJcXlOdDVQTGd3dlIvWk15TUl0Ym5tUlhmNFVWZ1NtRjY4ZDBxL3VmdnErQnpDaURSRFY5N1ZCSUxRTUFsR2dJWDQxbUFGcHoya2ZOS2ExZmNLa1lzS1FadVRDT21UNHBsTU5jOFkyTm9zRTlwb0d4OTc5T2tVcmZjeFg4M1h3anRRbUx2bmtBUFZuL3FXS1NVN01pZG04dDkvNkdTNWlYRS9NclZ6akVMT3ZZZXBFK1dpRGR6V1Z2ZFlOYmlGamc3QzcwM1ZON3BUWXRkUDAzZzJ3ZTlyUUJsbjgrU1cyK1YrL0tnZTl0YU1tVTBOQlF4QlAzVEVwcHRGcEZ3RnhqaWEwY3I2YjF1WWdaWnJGNjgzZnhUTkRxYlJNQlJsVklIV1FrK3NRWXJYZmRIYVVkbnZUTTBhYmF2SVJ2aVVqOTRFa2FZeXpzcVJLMjMwU0ZlUlI2THdjK2ttSkMxRnNFbG9UVjBTZExDelpMN1NRYmhSZVNWUUFxSTBwdXFZT0w2VXArK1ZMK3kxdEtGSU54eEM2V0NYaW5OTWl3dGVwT1ZPZ3dTemU1VE5xOU5Tc0ZjPQ==",
    'data-10-year' :"https://lvr.land.moi.gov.tw/SERVICE/QueryPrice/dc49f7b019067380c74fb4732b037dea?q=VTJGc2RHVmtYMStzQTFVTENVdzNnZ3VpMlBPTVp6OTBYUFNiSStKZ2VLbWhNSngvd3NPUWZlTHQ1VytQMnlZMGo2ZzgxVVR1bDFjRjdEcXc0TnNXdTM3c2F1d2lJN2hhUXJzQVYzbzNydnZWdnA5SmZieWpFUllBOVVuVXFBNHNHZURic3NmMnlaVmp1UWszVjhUY1E1VVorbEhCdjVHUmJtZG9qdFZ1NThSL1NySXNib3Y5YlVEN2Q3N3dyTTRGOTNKalZsQm5hcXdmV2xHYWM0eVo1cVgrdWxRSXdQZHJwOWpDbllYbUZGN3NqS0xmZTIvZ0J3c1BFajdHTWQzdkNzWkFFQ2JGSXJDNWRJdW1UVDZLYkd0TXJvK0JEWFlwcHlwWEd5ZWZEajNoY0IrMjFWd3ZzRG9qU1dNS0JENFlXOHhHN3dmbmxBYkdmWUliZ0NNbFFBWFJZcWtXSDBsZEZnL2NzeENwWStlaW9hMnJ3UDFGU0dQcWljeVdGUGI3L1hGVmdVNkpBZVJ0QVdqVG5paVRDQ3cyekx3V1ordGZVR2xRbXQ0VVpaZXoyTGJQZDBlVDI4UzhjSm9oV2N4YWxrM0ZnaGFyMFkyMzlQeDJqVzZxTzdyelAyTG1oTklIVVBZanJMcE10ZGZRNTd4cDZORDJVWldrdzNtdW1yRERYMTM3S2tpU1VTWVVjKzR3akRGSTFJMC84YkI1NFpwWGxOTWZJRllhOTJaUlFGK3RJQ1hyeTYrZU9ldVVDWE0wVTJVWUZKRHk1N20zU2xKR3hRVVp3MG1qSFk1RG9pQzY5VDc3N3dNOC9PRlZmMVFZcm04eGQ1NjZuZmYvWnpzZnRkOXQ5Qkhjbno2djV2TW10UjRucld4YW5wRHRaVlB1SmdCWUNUS0tmZkxMYk44WXZYVnZvNFkxTTh3bXk4b3hscEVRY3JtU0lOYjFEYnZ2WFhFVHNWR3FVYjRiOE03bmZzYzZNZldqRUxVPQ=="
}

Extract certain features

In [50]:
columns = ["lon", "lat", "g", "s", "b", "pu", "f", "v", "el", "m", "l", "e", "tp"]
new_columns = {
    "lon": "longitude",
    "lat": "latitude",
    "g": "house_age",
    "s": "area",
    "b": "property_type",
    "pu": "purpose",
    "f": "floor",
    "v": "layout",
    "el": "elevator",
    "m": "janitor",
    "l": "parking_space",
    "e": "closing_year",
    "tp": "price",
}

print("Start")

res = requests.get(urls[output_file_name])
data = res.json()

df = pd.DataFrame(data)
df.to_csv(f'{output_file_name}-raw.csv', index=False, encoding="utf_8_sig")

print("Finish")

Start
Finish


In [51]:
df = df[columns]
df = df.replace('', np.nan)
df = df.rename(columns=new_columns)
df[:5]

,longitude,latitude,house_age,area,property_type,purpose,floor,layout,elevator,janitor,parking_space,closing_year,price
0,120.977307,24.820037,32,26.41,華廈(10層含以下有電梯),住家用,三層/五層,3房2廳2衛,有,無,1,112/10/03,"9,250,000"
1,121.012857,24.790343,NaN,32.96,住宅大樓(11層含以上有電梯),住家用,二十二層/二十九層,2房2廳1衛,有,有,1,112/10/01,"8,000,000"
2,120.968306,24.776166,19,15.26,華廈(10層含以下有電梯),住家用,地下三層/四層,1房1廳1衛,有,有,0,112/10/02,"4,150,000"
3,120.931959,24.838812,28,60.74,透天厝,住家用,全/四層,5房2廳4衛,無,無,0,112/10/01,"13,550,000"
4,121.025405,24.778577,25,52.62,住宅大樓(11層含以上有電梯),住家用,七層/十四層,3房2廳2衛,有,有,1,112/10/03,"16,600,000"


Handle the missing values

In [52]:
# print out the column which contains missing values 
missing_columns = []
for column_name in df.columns:
    missing_count = df[column_name].isna().sum()
    print(f'{column_name}: {missing_count}')
    if missing_count > 0:
        missing_columns.append(column_name)

longitude: 0
latitude: 0
house_age: 20976
area: 0
property_type: 1
purpose: 53
floor: 4
layout: 2266
elevator: 39982
janitor: 0
parking_space: 0
closing_year: 0
price: 0


In [53]:
for column_name in missing_columns:
    mode_value = df[column_name].mode()[0]
    df[column_name].fillna(mode_value, inplace=True)

    missing_count = df[column_name].isna().sum()
    print(f'{column_name}: {missing_count}')

house_age: 0
property_type: 0
purpose: 0
floor: 0
layout: 0
elevator: 0


Layout processing

In [54]:
df[['bedroom', 'living_room', 'bathroom']] = df['layout'].str.extract('(\d+)房(\d+)廳(\d+)衛')
df[['bedroom', 'living_room', 'bathroom']] = df[['bedroom', 'living_room', 'bathroom']].fillna(0).astype(int)
df = df.drop(columns=['layout'])

In [55]:
df[['bedroom', 'living_room', 'bathroom']]

,bedroom,living_room,bathroom
0,3,2,2
1,2,2,1
2,1,1,1
3,5,2,4
4,3,2,2
...,...,...,...
65230,4,2,3
65231,4,2,3
65232,4,2,3
65233,4,2,4


`Elevator`, `Janitor` and `purpose` processing (Label Encoding)

In [56]:
binary_mapping = {"有": 1, "無": 0}
df["elevator"] = df["elevator"].map(binary_mapping)
df["janitor"] = df["janitor"].map(binary_mapping)

df["residential"] = df["purpose"].map(lambda val : 1 if val == '住家用' else 0)
df = df.drop(columns=['purpose'])

In [57]:
df[['elevator', 'janitor', 'residential']]

,elevator,janitor,residential
0,1,0,1
1,1,1,1
2,1,1,1
3,0,0,1
4,1,1,1
...,...,...,...
65230,1,1,1
65231,1,1,1
65232,1,1,1
65233,1,1,1


`Property Type` processing (One Hot Encoding)

In [58]:
property_type_mapping = {
    '住宅大樓(11層含以上有電梯)': 'residential_building',
    '公寓(5樓含以下無電梯)': 'apartment',
    '華廈(10層含以下有電梯)': 'mansion',
    '透天厝': 'detached_house'
}

df['property_type'] = df['property_type'].map(property_type_mapping)
df = pd.get_dummies(df, columns=['property_type'])

In [59]:
df[['property_type_residential_building', 'property_type_apartment', 'property_type_mansion', 'property_type_detached_house']]

,property_type_residential_building,property_type_apartment,property_type_mansion,property_type_detached_house
0,0,0,1,0
1,1,0,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
...,...,...,...,...
65230,1,0,0,0
65231,1,0,0,0
65232,1,0,0,0
65233,1,0,0,0


`Floor` processing 

In [60]:
# Split floor to sale_floor and total_floor
df[['sale_floor', 'total_floor']] = df['floor'].str.split('/', expand=True)
df = df.drop(['floor'], axis=1)

In [61]:
# Convert Chinese to Arabic number in total_floor column
df['total_floor'] = df['total_floor'].replace('層','', regex=True)
df = df.drop(df[df['total_floor'] == '--'].index)    # Remove the row if the total_floor value is '--'
df['total_floor'] = df['total_floor'].apply(lambda x: cn2an.cn2an(x))


In [62]:
# After splitting floor, total_floor or sale_floor might be empty  
df = df.replace('', np.nan)
for column_name in ['total_floor', 'sale_floor']:
    mode_value = df[column_name].mode()[0]
    df[column_name].fillna(mode_value, inplace=True)

In [63]:
# Split sale_floor to sale_floor, sale_floor_basement, sale_floor_all, sale_floor_other 
def sale_floor_mapping(value):
    keywords = ['騎樓', '見其他登記事項', '夾層', '屋頂突出物', '儲藏室', '避難室兼停車場', '防空避難室']
    if any(keyword in value for keyword in keywords):
        return 'other'
    elif '地下' in value:
        return 'basement'
    elif '全' in value:
        return 'all'
    else:
        value = value.split(',')[0].replace('層', '')
        return cn2an.cn2an(value)

df['sale_floor'] = df['sale_floor'].apply(sale_floor_mapping)
df['sale_floor_all'] = df['sale_floor'].apply(lambda x: 1 if x == 'all' else 0)
df['sale_floor_basement'] = df['sale_floor'].apply(lambda x: 1 if x == 'basement' else 0)
df['sale_floor_other'] = df['sale_floor'].apply(lambda x: 1 if x == 'other' else 0)

# Convert Chinese to Arabic number in sale_floor column
df['sale_floor'] = df['sale_floor'].apply(lambda x: x if isinstance(x, (int)) else 0)
df = pd.concat([df.drop('sale_floor', axis=1), df['sale_floor']], axis=1)

In [64]:
df[['sale_floor_all', 'sale_floor_basement', 'sale_floor_other', 'sale_floor', 'total_floor']]

,sale_floor_all,sale_floor_basement,sale_floor_other,sale_floor,total_floor
0,0,0,0,3,5
1,0,0,0,22,29
2,0,1,0,0,4
3,1,0,0,0,4
4,0,0,0,7,14
...,...,...,...,...,...
65230,0,0,0,8,14
65231,0,0,0,10,14
65232,0,0,0,3,14
65233,0,0,0,13,14


`Closing year` processing

In [65]:
df['closing_year'] = df['closing_year'].str.split('/', expand=True)[0]

In [66]:
df['closing_year']

0        112
1        112
2        112
3        112
4        112
        ... 
65230    102
65231    102
65232    102
65233    102
65234    102
Name: closing_year, Length: 65214, dtype: object

Move `sale_floor` and `total_floor` back

In [67]:
df = pd.concat([df.drop('sale_floor', axis=1), df['sale_floor']], axis=1)
df = pd.concat([df.drop('total_floor', axis=1), df['total_floor']], axis=1)

`Price` processing 

In [68]:
# Price string to int
df['price'] = df['price'].replace(',','', regex=True).map(int)
# move price back
df = pd.concat([df.drop('price', axis=1), df['price']], axis=1)
# drop (不知道哪個 RZ 在上面登記 0 元的房子)
df.drop(df[(df['price'] == 0)].index, inplace=True)

In [69]:
df['price']

0         9250000
1         8000000
2         4150000
3        13550000
4        16600000
           ...   
65230    16800000
65231    17000000
65232    16790000
65233    27800000
65234     4950000
Name: price, Length: 65213, dtype: int64

Output the file

In [70]:
df.to_csv(f'{output_file_name}.csv', index=False, encoding="utf_8_sig")
df

,longitude,latitude,house_age,area,elevator,janitor,parking_space,closing_year,bedroom,living_room,...,property_type_apartment,property_type_detached_house,property_type_mansion,property_type_residential_building,sale_floor_all,sale_floor_basement,sale_floor_other,sale_floor,total_floor,price
0,120.977307,24.820037,32,26.41,1,0,1,112,3,2,...,0,0,1,0,0,0,0,3,5,9250000
1,121.012857,24.790343,1,32.96,1,1,1,112,2,2,...,0,0,0,1,0,0,0,22,29,8000000
2,120.968306,24.776166,19,15.26,1,1,0,112,1,1,...,0,0,1,0,0,1,0,0,4,4150000
3,120.931959,24.838812,28,60.74,0,0,0,112,5,2,...,0,1,0,0,1,0,0,0,4,13550000
4,121.025405,24.778577,25,52.62,1,1,1,112,3,2,...,0,0,0,1,0,0,0,7,14,16600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65230,120.962220,24.820079,1,86.82,1,1,4,102,4,2,...,0,0,0,1,0,0,0,8,14,16800000
65231,120.962220,24.820079,1,86.82,1,1,4,102,4,2,...,0,0,0,1,0,0,0,10,14,17000000
65232,120.962220,24.820079,1,86.82,1,1,4,102,4,2,...,0,0,0,1,0,0,0,3,14,16790000
65233,120.962220,24.820079,1,117.14,1,1,4,102,4,2,...,0,0,0,1,0,0,0,13,14,27800000
